# Introduction/Business Problem
### A description of the problem and a discussion of the background.
Taking on the role of a restruarantier; I will use my new data science knowledge to help in the location planning phase for a new venture in a wealthy, fast growing city inside the United States.  

# Data

### A description of the data and how it will be used to solve the problem.

To solve this challenge, the following preliminary plan is in place.
1. Determine the proper cities/areas which are both wealthy and fast growing.
2. Determine which eateries in the chosen area receive the most visitors.
3. Identify a location within the area with the largest gap between similar restaurants which still holds other venues with high foot traffic
4. Decide to either accept the location, or to iterate through these steps until a suitable location has been identified.


In support of that plan, the following data sources will be used.
Census data:
* The expected wealth of populations in a city
* The recent growth rate of the cities being reviewed

Foursquare:
* Review venues to identify the popularity of differing types of restaurants (comparing foot traffic to other restaurants)
* Foot traffic experienced by other businesses nearby potential locations

Other:
* Geo-spacial location data from cocl.us will be used for visualizations

All of the data will be compiled and appropriate visualizations will be produced and reviewed.  A final determination will be made by hand.